<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/tapi-logo-small.png" />

This notebook free for educational reuse under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

Created by J.D. Porter  for the 2024 Text Analysis Pedagogy Institute, with support from [Constellate](https://constellate.org).
<br />
____

# `Small Language Models` `1`

This is lesson `1` of 3 in the educational series on `small language models`. This notebook is intended `to teach very basic text mining skills and introduce the concepts of distributional word meaning that underly some recent developments in large language models (LLMs) and AI`. 

**Skills:** 
* Text analysis
* Python

**Audience:** `Teachers` / `Learners` / `Researchers`

**Use case:** `Tutorial`

**Difficulty:** `Beginner`

**Completion time:** `90 minutes`

**Knowledge Required:** 
```
* Python basics (variables, functions, lists, dictionaries)
```

**Knowledge Recommended:**
```
n/a
```

**Learning Objectives:**
After this lesson, learners will be able to:
```
1. Open a text file and convert it to a list of words
2. "Clean" the words using a function
3. Find counts and frequencies for the words
4. Filter unwanted words
5. Repeat this process for every text file in a directory
```
**Research Pipeline:**
```
If you want to try some of this on your own, you can:
1. Gather some texts and save them on your machine as .txt files
2. Use whatever steps you're interested in from this notebook
3. Interpret!
```
___

# Required Python Libraries
 * To keep things simple, we will try to work with very few libraries in this notebook (just 'os', which you do not need to install, though you do need to import it)

In [1]:
### Import Libraries ###
import os

# Required Data

**Data Format:** 
* plain text (.txt)

**Data Source:**
* included files (though you may supplement these whenever you feel comfortable doing so!)

**Data Quality/Bias:**

Included texts are from freely available sources like Project Gutenberg and Wikipedia. They have not been vetted for textual accuracy relative to, say, a scholarly edition.

**Data Description:**

This lesson uses textual data in .txt format (utf-8 encoding) from various sources.

In [ ]:
### Grab files with console `wget` and `mv` ###
!wget https://github.com/tesseract-ocr/tessdata/raw/main/eng.traineddata
!mv eng.traineddata /usr/share/tesseract-ocr/4.00/tessdata/eng.traineddata


In [ ]:
### Grab a single file and supply name ###
urllib.request.urlretrieve('https://file.address.txt', 'filename.txt')

In [ ]:
### Retrieve multiple files using a list and string splitting###

download_urls = [
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/sample_01.pdf',
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/sample_02.pdf',
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/sample_03.pdf'
]

for url in download_urls:
    urllib.request.urlretrieve(url, url.rsplit('/', 1)[-1])

In [ ]:
### Retrieve multiple files using a list and Path ###
from pathlib import Path
import urllib.request

# Check if a folder exists to hold pdfs. If not, create it.
pdfs_folder = Path.cwd() / 'data' / 'sample_pdfs'
pdfs_folder.mkdir(parents=True, exist_ok=True)

# Define a list of URLs for our sample pdfs. 
download_urls = [
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/sample_01.pdf',
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/sample_02.pdf',
    'https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/sample_03.pdf'
]

# For each URL, download the file and put it into the folder with the original name
for url in download_urls:
    path_url = Path(url)
    urllib.request.urlretrieve(url, f'{pdfs_folder.as_posix()}/{path_url.name}')
    
## Success message
print('Folder created and pdfs added.')

In [ ]:
### Constellate Example ###

# Importing your dataset with a dataset ID
import constellate
# Pull in the sampled dataset (1500 documents) that matches `dataset_id`
# in the form of a gzipped JSON lines file.
# The .get_dataset() method downloads the gzipped JSONL file
# to the /data folder and returns a string for the file name and location
dataset_file = constellate.get_dataset(dataset_id)

# To download the full dataset (up to a limit of 25,000 documents),
# request it first in the builder environment. See the Constellate Client
# documentation at: https://constellate.org/docs/constellate-client
# Then use the `constellate.download` method show below.
#dataset_file = constellate.download(dataset_id, 'jsonl')


# Introduction...

## ... to the course

This course focuses on the idea that we can capture information about the meanings of words by measuring how they go together in texts.  This is one of the core concepts behind large language models (LLMs), which are in turn a core component of the kinds of generative AI that have captured a lot of attention lately—models like OpenAI or Llama that can produce surprisingly complex and “human-like” texts. 

Whether we’re talking about these AI approaches in general or LLMs in particular, the underlying technology is quite complex. Some elements involve high level technical training that most of us won’t (and maybe can’t) realistically get. Other elements are often described as “black boxes”, opaque even to specialists who have that training. That’s to be expected for technology this advanced and powerful (how many of us *really* understand the COVID vaccine, or the space shuttle?), but in this case the stakes of not knowing seem high. These systems are advertised as powerful forms of “intelligence” that may replace any number of human activities, and may even approach becoming humanity. Separating the hype from the very real advances has high stakes! So we may want to take any form of demystification we can get. 


That’s the idea behind this course. For all their complexity, contemporary AI systems still depend to a significant extent on a basic idea: that if we measure how words appear near each other in some corpus of language, we can model how those words come to mean things. We will work together to create our own simple vector models of a small corpus. These will enable us to see for ourselves what happens as we change our texts, adjust parameters, and try out methods of streamlining our code. We won’t wind up with chat bots that can write a **B-** college essay about the Byzantine empire on command. But we will gain some insight into how such a chat bot comes to understand which words an essay about Byzantine empire should use. And along the way, we’ll pick up a few skills in text mining in Python. But first, let’s take a quick look at the idea of measuring word meaning “distributionally.”

## ... to LLMs

Before we get to the code, it's worth zooming out a bit to get some perspective on language models. The term does not have a precise definition, and even the phrase "Large Language Models" (LLMs) gets attached to different kinds of objects. For our purposes, we can just adapt the definition you get when you look up [LLMs on Wikipedia](https://en.wikipedia.org/wiki/Large_language_model).

    An LLM is a computational model that can do some level of language generation and other natural language processing tasks. Based on language models, LLMs acquire these abilities by learning statistical relationships from vast amounts of text during a computationally intensive training process. LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.

Already you might notice that large language models are "based on language models", which is a good example of the lack of clarity surrounding these terms. So let's try to break down a few components of LLMs, and hopefully work our way around to language models more generally.

The rapid advances we've seen in LLMs over the past few years involve four fundamental factors. Three of these aren't really useful for us this week, but they're worth mentioning briefly, since they help with our goal of demystifying LLMs.
 1. **Vast Amounts of Text** <br> One of the most well-known features of models like ChatGPT is that they involve gigantic datasets. Sometimes people describe them as having "read the Internet". This isn't exactly true, but the scale is still mind-boggling. Stats in this area are usually reported in terms of the number of tokens, which isn't equivalent to the number of words, but isn't too far off. For example, this sentence <br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *Ships at a distance have every man's wish on board.*</br></br> has 13 tokens according to [Open AI's tokenizer tool](https://platform.openai.com/tokenizer). It also has 10 words, so same order of magnitude. Proust's *In Search of Lost Time* is often considered the longest major novel ever written, and it contains about 1 million words. The Gale American Fiction corpus, which contains around 18,000 novels, consists of 1.2 billion words. All of Wikipedia in English is 4.3 billion words. GPT-3 supposedly contains *499 billion* tokens—one hundred times the size of all of English Wikipedia, and that's not even the latest model, which may be significantly bigger. So that's a lot of text.</br><br></br>
 2. **Neural Nets** <br> Neural nets play an important role in the machine learning processes used by LLMs. They're notoriously complex, but I like this description from the director of the Stanford Literary Lab, Mark Algee-Hewitt:<br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"A neural network is a computational model that is trained on a set of variables to classify objects. Unlike most models, a neural network does not use a derived set of formulae (as in a discriminant function analysis) or probabilistic model (as in a topic model), but instead seeks to simulate a reasoning process through a system of virtual neurons and synapses (created by the algorithm as a set of nodes and edges). Fed a list of variable inputs and set of known classifications, a neural network trains itself, using a feedback look and a set of hidden operations to create a complex internal system of pattern recognition that can be used to classify future objects computationally. More powerful and flexible than other kinds of algorithmic models, the neural network exchanges reliability for interpretability, a simulated mind, but one whose internal logic is impenetrable."[$^{1}$](#1)</br></br> When I worked at the Literary Lab, I was part of a team that was studying literary suspense. As part of that, we wanted to see if we could train a neural net to identify suspenseful passages. So, we humans on the team read a bunch of passages and tagged them for their suspensefulness. We showed the model 60% of those passages, along with some information about the word frequencies and distributions in our corpus. The neural net then tried to figure out how to use that information to correctly tag passages as suspenseful or not. It eventually got up to 92% accuracy on passages it had seen before, and more impressively, 81% accuracy on passages it hadn't seen.</br><br>We really want to take away two things here. First, neural nets are kind of astonishingly effective. But, second, they're generally considered "black boxes". That is, they can't really be reverse engineered. The iterative process makes it nearly impossible to know exactly *how* the algorithm reaches its conclusions. The "internal logic is impenetrable", as Algee-Hewitt says.</br><br></br>
 3. **Transformer Architecture**<br> The development of transformer, or “self-attention” methods, really kicks off in 2017, which, if you're keeping score, is right about the time we started getting the kinds of breakthrough models that we have today. A research team at Google published a paper about the idea, and it's probably not too out of line to call it one of the most influential scientific papers of the century so far.[$^{2}$](#2) For a bit of context: In my world, Sianne Ngai's *Theory of the Gimmick* is a really important book. It has 75 citations on Google Scholar, which is a great showing for something that came out in 2017. But that Google paper came out the same year, and it has about 125,000 citations. Who knows how many are from humans, but still. In any case, the concept is once-again complex for non-specialists, but the basic idea is that transformers enable models to pass contextual information up the analytical chain with much better efficiency and greater success than previous methods attained. I think Timothy B. Lee and Sean Trott have a [nice, accessible overview](https://www.understandingai.org/p/large-language-models-explained-with) of the idea if you want to go deeper. I mention it because it's important, and because it underscores the complexity of LLMs.</br><br></br>
 4.**The Distributional Hypothesis**<br> Finally we get to the part of LLMs that is really important for us this week. On a general level, the idea is that if you know how words are distributed across a big corpus of language, you can determine their semantic properties (and possibly other components of meaning as well). You can find important versions of this concept in, for instance, the linguistic approach of Ferdinand de Saussure, or in the later philosophy of Ludwig Wittgenstein. But we'll focus on the version that comes through linguistics starting in roughly the 1950's; if you teach this, you're required to quote the linguist J.R. Firth: “You shall know a word by the company it keeps.”</br><br>Word vectors are one way to operationalize the distributional hypothesis. Let's imagine we have a corpus of texts, and we look up how often the words "penguin", "duck", and "eagle" appear near the word "flies":

        penguin                     duck             eagle
    ------------------------------------------------------>
                        "flies"

This shows that "penguin" appears near "flies", let's say, 1 time, "duck" 8 times, and "eagle" 10. On this single dimension, it looks like ducks and eagles are pretty similar, and penguins not so much. Now let's add a dimension showing how often they appear near "swims":

     |
     |
     |  penguin
     |
    "|
    s|                              duck
    w|
    i|
    m|
    s|
    "|
     |
     |
     |                                               eagle
     | 
     ------------------------------------------------------>
                        "flies"

We can measure distance here, too—for instance, we can just draw a straight line from one word's location to the next. We'll talk more about ways to measure distance later, but for now the point is that adding this new dimension helps us see that "duck" might actually be somewhat close to "penguin", while eagle may be farther from "duck" than we first thought.

I can't draw it in markdown very easily, but we can imagine adding a z-axis, turning our little graph into a 3-D box. If the z-axis shows us how often each word appears near "hockey", then "duck" and "penguin" would be way at the back of the box (since there are hockey teams called the Ducks and the Penguins), whereas eagle would be up at the front of the box (they've got a football team (go birds), but not hockey). And now "duck" and "penguin" would be much closer to each other than either would be to "eagle".

As you can imagine, if we keep adding dimensions, we'll learn more and more about how these words relate to each other. We humans can't picture any more dimensions than three, but we can pretty easily calculate the distance between words in higher dimensional arrangements.

A word vector is basically just another way to show the information captured in these graphs. Let's work our way up to it. Here's a table showing the data about how each of our bird words relates to "flies", "swims", and "hockey".

| term | "flies" | "swims" | "hockey" |
| ---- | ---- | ---- | ---- |
|penguin | 1 | 9 | 9
|duck | 8 | 7 | 8
|eagle | 10 | 1 | 0

We can strip that down a bit to show just the terms and the numbers associated with them.

penguin 1 9 9<br> 
duck 8 7 8 <br>
eagle 10 1 0 <br>

And we can even go a step farther, and render each term as the list of numbers (we'll format these lists the way we make lists in Python).

```Python
penguin = [1,9,9]
duck = [8,7,8]
eagle = [10,1,0]
```

These lists of numbers are word vectors. "Vector" in this context just means "a list of numbers".

Remember that as we add dimensions (e.g., checking how often each term appears near "lake", "ice", "dollar", "daffy", "macaroni", etc.), our lists will get longer, and they'll more precisely situate each of our terms. That means they'll do a better and better job capturing the *meaning* of each term. After all, what is a duck if not a bird that flies and swims near lakes, represents a hockey team, and sometimes pauses to argue with Bugs Bunny? 

If you do this process at a large scale, you get a *model* of the way all of your words fit together. That's a **language model**. Specifically, it's a vector model. You can use it to see the most similar words to any particular word, to see how words relate to each other, to predict likely next words in a sentence, and so on.

These models can quickly become extremely unwieldy. If you have one billion words measured against one billion other words, it's going to take a lot of computing power to work with them. Some of the complexity and inscrutability of LLMs comes down to issues of scale (point 1, above). Some of it comes down to the methods used to render that scale computationally tractable, or to improve the probabilistic intepretation of the data (points 2 and 3). But ultimately, LLMs are based on the distributional hypothesis. And that's the part of all of this that we can simplify, understand, and make ourselves. We'll do that by Friday.

<hr></hr>

##### *Footnotes* #####

1. <a id="1"></a> This comes from an unpublished essay about the project. Cited here with the author's permission. other members of the suspense project include Abigail Droge, Tasha Eccles, Morgan Day Frank, Erik Fredner, Andrew Shephard, and Hannah Walser.
2. <a id="2"></a> Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems 30 (2017). [Link here](https://proceedings.neurips.cc/paper_files/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html)

## ... to this lesson

Before we get to word vector models, we need to know some basics about working with texts. That's what we'll focus on today. Some of this material might feel a long way off from our end goal, but starting a little slowly will help us ramp up sooner than you might expect. So, we're going to learn how to open a text, get the words out, clean them up a little, and count them. The aim here is to give you very basic, flexible, lightweight tools, usable with a pretty minimal level of familiarity with Python.

The section headings below tell the story the rest of the way... so let's dive in!

# From files to words


In [ ]:
# Open a file the Pythonic way

# First, we point to the file


In [ ]:
# Next, we open the file and read it


In [ ]:
# Finally, we split the text into words


# Counting the words

In [9]:
# First, set up a dictionary. It will take words as keys and store counts as values (though for now it's empty)

# Next, use a for loop to iterate through our words, counting them as we go
# Remember that you can use n += x to increment some variable n by amount x

# Notice any problems with our results? That's why we need the next section!

# Cleaning the Words

In [6]:
# Run this cell to get a useful example sentence

sentence = 'She said, "We should leave," and so - with some reservations - we did.'

In [7]:
# We'll build our function here


In [8]:
# We'll test our function here


# Working with a Directory

In [ ]:
# Get the files


In [ ]:
# Add the path name


In [ ]:
# Use a for loop to repeat the steps we took for one single file


# On Zipf's Law

Zipf's Law, named for the linguist George Zipf, who figured it out in the 1930's, basically says that if you arrange the words in a text in frequency order, the first one will occur about twice as often as the second one, which will occur about twice as often as the third, and so on. The exact mathematical relationship is not all that important for our purposes (it's not always that clean), but the gist is: The most common words take up a huge percentage of the text, and there's a "long tail" of words that appear just once (or only a few times).

In [ ]:
# Checking out some statistics about our words

# Getting the total word count of our corpus

# Seeing what percentage of words are "the"

# Sorting the dictionary
# This is a little annoying to do, so I've written it out ahead of time
s = sorted(counts,key = lambda i:counts[i],reverse=True)
sorted_counts = [(i,counts[i]) for i in s]

# Seeing how much of the corpus consists of the x most common words

# Seeing how many words occur just x number of times